# **CREATING THE FINAL PANEL**

In [3]:
import param
import panel as pn
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from bokeh.plotting import figure
import networkx as nx
import matplotlib.pyplot as plt
from math import pi
from bokeh.palettes import Category20c, Category20, Category10
from bokeh.plotting import figure
from bokeh.transform import cumsum
import numpy as np
import io
import torch

ModuleNotFoundError: No module named 'torch'

In [4]:
train_set = pd.read_csv('train_set.csv')
test_set = pd.read_csv('test_set.csv')
df_edges_init = pd.read_csv('elliptic_bitcoin_dataset/elliptic_txs_edgelist.csv')

In [5]:
# Obtain trained random forest model
def train_random_forest(train_set, test_set):
    X_train = train_set.loc[train_set['class'].isin([1, 2])].drop(columns=['class'])
    y_train = train_set.loc[train_set['class'].isin([1, 2])]['class']
    
    X_test = test_set.loc[test_set['class'].isin([1, 2])].drop(columns=['class'])
    y_test = test_set.loc[test_set['class'].isin([1, 2])]['class']
    
    model = RandomForestClassifier()

    return model.fit(X_train, y_train)

random_forest_model = train_random_forest(train_set, test_set)
# AE Embeddings, AE reconstruction

In [1]:
class SAGE(torch.nn.Module):
    def __init__(self, in_channels,
                hidden_channels, out_channels,
                n_layers=2):
        
        super(SAGE, self).__init__()
        self.n_layers = n_layers
        self.layers = torch.nn.ModuleList()
        self.layers_bn = torch.nn.ModuleList()
        if n_layers == 1:
            self.layers.append(SAGEConv(in_channels, out_channels,   normalize=False))
        elif n_layers == 2:
            self.layers.append(SAGEConv(in_channels, hidden_channels, normalize=False))
            self.layers_bn.append(torch.nn.BatchNorm1d(hidden_channels))
            # self.layers.append(SAGEConv(hidden_channels, out_channels, normalize=False))
        else:
            self.layers.append(SAGEConv(in_channels, hidden_channels, normalize=False))
            # self.layers_bn.append(torch.nn.BatchNorm1d(hidden_channels))
        for _ in range(n_layers - 2):
            self.layers.append(SAGEConv(hidden_channels,  hidden_channels, normalize=False))
            self.layers_bn.append(torch.nn.BatchNorm1d(hidden_channels))
                    
                
        if n_layers != 1:
            self.layers.append(SAGEConv(hidden_channels, out_channels, normalize=False))
        
        for layer in self.layers:
            layer.reset_parameters()
            
            
            
    def forward(self, x, edge_index):
        if len(self.layers) > 1:
            looper = self.layers[:-1]
        else:
            looper = self.layers
        
        for i, layer in enumerate(looper):
            x = layer(x, edge_index)
            # print(f"SHAPE: {x.shape}, step: {i}")
            # print(f"Step: {i}")
            try:
                x = self.layers_bn[i](x)
            except Exception as e:
                abs(1)
            finally:
                x = F.relu(x)
                x = F.dropout(x, p=0.5, training=self.training)
        
        if len(self.layers) > 1:
            x = self.layers[-1](x, edge_index)
        return F.log_softmax(x, dim=-1), torch.var(x)
    
    def inference(self, total_loader, device):
        xs = []
        var_ = []
        for batch in total_loader:
            out, var = self.forward(batch.x.to(device), batch.edge_index.to(device))
            out = out[:batch.batch_size]
            xs.append(out.cpu())
            var_.append(var.item())
        
        out_all = torch.cat(xs, dim=0)
        
        return out_all, var_

NameError: name 'torch' is not defined

In [ ]:
# Obtain trained GNN, 
def get_GNN_model(path, input):

    #carreguem el model
    model = SAGE()
    model.load_state_dict(torch.load(path))
    model.eval()
    
    input_tensor = input
    with torch.no_grad():
        output = model(input_tensor)
    
    return output
    
gnn_model = get_GNN_model(path="GNN_models/trained_models/final_model_2_layers.pth", input=X_train)

In [9]:
class dashboard:
    def __init__(self, train_set, test_set, df_edges_init, random_forest_model):# GNN_model):#AE_emb_model, AE_rec_model):
        self.train_set = train_set
        self.test_set = test_set
        self.df_edges_init = df_edges_init
        self.df_numerical_results = pd.DataFrame(columns=["Licit", "Ilicit"])
        self.rf_model = random_forest_model
        # self.GNN_model = GNN_model
        #self.AE_emb_model = AE_emb_model
        #self.AE_rec_model = AE_rec_model
        
        
    def display_df_predictions(self, timestep, model):
        # Create a subdataframe based on the timestep
        if self.train_set[self.train_set["Time Step"] == timestep].empty:
            self.df_subgraph = self.test_set[self.test_set['Time Step'] == timestep]
        else:
            self.df_subgraph = self.train_set[self.train_set['Time Step'] == timestep]
        self.df_edges = self.df_edges_init.loc[(self.df_edges_init['txId1'].isin(self.df_subgraph['txId'])) & (self.df_edges_init['txId2'].isin(self.df_subgraph['txId']))]
        
        # Obtain the predictions from the trained model chosen by the user
        X_test = self.df_subgraph.loc[self.df_subgraph['class'].isin([1, 2, 3])].drop(columns=['class'])
        
        if model == "Random Forest":
            self.y_pred = self.rf_model.predict(X_test)
            
        elif model == "GNN":
            self.y_pred = self.GNN_model.predict(X_test)
            
        elif model == "Autoencoders Embeddings":
            pass
            
        else:
            pass


        # Store the predictions of the model in a dataframe
        # Only display the id node, time step the class
        self.df_predictions = self.df_subgraph.iloc[:, 0:3]
        # Rename some columns
        columns_to_replace = {'index': 'Index', 'txId': 'Id Node', 'class': 'True Class'}
        self.df_predictions.rename(columns=columns_to_replace, inplace=True)
        # Insert the predictions
        self.df_predictions.insert(loc=2, column='Predicted Class', value=self.y_pred)
        # Replace the numbers for licit, ilicit or unknown
        df_final_predictions = self.df_predictions.replace({1: "Ilicit", 2: "Licit", 3: "Unknown"})

        # # Add filters for the user
        # filters = {
        #     'True Class': {'type': 'list', 'func': 'in', 'valuesLookup': True, 'sort': 'asc', 'multiselect': True},
        #     'Predicted Class': {'type': 'list', 'func': 'in', 'valuesLookup': True, 'sort': 'asc', 'multiselect': True},
        # }

        
        # filter_table = pn.widgets.Tabulator(
        #     df_final_predictions, pagination='local', layout='fit_columns', page_size=4, sizing_mode='stretch_width',
        #     header_filters=filters
        # )

        return df_final_predictions


    def summary_results(self):
        # Create a dataframe of number of licit and ilicit nodes of predicted and true labels
        licit = {
            "True Label": self.df_predictions['True Class'].value_counts()[2],
            "Predicted Label": self.df_predictions['Predicted Class'].value_counts()[2]
        }
        
        ilicit = {
            "True Label": self.df_predictions['True Class'].value_counts()[1],
            "Predicted Label": self.df_predictions['Predicted Class'].value_counts()[1]
        }
        
        summary = {
            "Licit": licit,
            "Ilicit": ilicit
        }

        self.df_summary_results = pd.DataFrame(summary)

        return pn.widgets.Tabulator(self.df_summary_results)


    def display_pie_charts(self, label):
        if self.df_summary_results is None:
            self.summary_results()
        # Pie chart for true labels
        if label=="true":
            labels = self.df_summary_results.loc["True Label"]
            data = labels.reset_index(name='value').rename(columns={'index':'TrueLabel'})
            
            # Pie chart
            data['angle'] = data['value']/data['value'].sum() * 2*pi
            data['color'] = Category10[10][2:4]
            
            p = figure(height=350, title="True Labels", toolbar_location=None,
                       tools="hover", tooltips="@TrueLabel: @value", x_range=(-0.5, 1.0))
            
            r = p.wedge(x=0, y=1, radius=0.4,
                    start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
                    line_color="white", fill_color='color', legend_field='TrueLabel', source=data)
            
            p.axis.axis_label=None
            p.axis.visible=False
            p.grid.grid_line_color = None


        # Pie chart for predicted label
        else:
            labels = self.df_summary_results.loc["Predicted Label"]
            data = labels.reset_index(name='value').rename(columns={'index':'PredictedLabel'})

            # Pie chart
            data['angle'] = data['value']/data['value'].sum() * 2*pi
            data['color'] = Category10[10][2:4]
            
            p = figure(height=350, title="Predicted Labels", toolbar_location=None,
                       tools="hover", tooltips="@PredictedLabel: @value", x_range=(-0.5, 1.0))
            
            r = p.wedge(x=0, y=1, radius=0.4,
                    start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
                    line_color="white", fill_color='color', legend_field='PredictedLabel', source=data)
            
            p.axis.axis_label=None
            p.axis.visible=False
            p.grid.grid_line_color = None

        
        return pn.pane.Bokeh(p, theme="dark_minimal")
        
    
    def display_graph(self, label):
        # Create graph
        graph = nx.Graph()
        
        for _, row in self.df_predictions.iterrows():
            # Extract node ID and attributes
            node_id = row['Id Node']
            node_attributes = row.drop('Id Node').to_dict()
            
            # Add node to the graph with its attributes
            graph.add_node(node_id, **node_attributes)

        # Add edges
        for _, row in self.df_edges.iterrows():
            graph.add_edge(row['txId1'], row['txId2'])

        
        # Display graph
        fig,ax=plt.subplots()
        pos = nx.spring_layout(graph)
        
        if label=="true": # True graph
            colors = {1: 'red', 2: 'green', 3: 'black'}
            nx.draw(graph, pos, ax=ax,with_labels=False, node_color=[colors[graph.nodes[n]['True Class']] for n in graph.nodes()], node_size=20, font_size=12)
        else: # Predicted graph
            colors = {1: 'red', 2: 'green'}
            nx.draw(graph, pos, ax=ax,with_labels=False, node_color=[colors[graph.nodes[n]['Predicted Class']] for n in graph.nodes()], node_size=20, font_size=12)
        
        return fig



db = dashboard(train_set, test_set, df_edges_init, random_forest_model)

In [12]:
pn.extension('tabulator', design='material', loading_spinner='dots', loading_color='#00aa41', template='bootstrap', notifications=True)
pn.widgets.Tabulator.theme = 'materialize'
pn.param.ParamMethod.loading_indicator = True

#*****************************************************************************************************
# For tab 1: create buttons to enter the text and upload the file, also a button to clear the history
#*****************************************************************************************************
# file_input = pn.widgets.FileInput(accept='.pkl')
# button_load = pn.widgets.Button(name="Load file", button_type='primary')
# # button_load.param.watch(interactions.load_file(), 'clicks')
# # bound_button_load = pn.bind(interactions.load_file, button_load.param.clicks)

# upload_file_box = pn.WidgetBox('## Upload file', file_input, button_load)

# tab1 = pn.Column(
#     pn.layout.Divider(),
#     pn.Row(),
#     pn.Row(upload_file_box),
#     pn.Row(),
#     pn.Row(file_input.value),
#     pn.layout.Divider(),
# )

#********************************************************************************************
# Tab 2: Creating options for the user to choose from and displaying the predictions obtained
#********************************************************************************************
timestep_widget = pn.widgets.IntSlider(name="TimeStep", value=30, start=1, end=49)
model_widget = pn.widgets.RadioButtonGroup(name="Models", options=["Random Forest", "GNN", "Autoencoders Embeddings", "Autoencoders Reconstruction"])

predictions = pn.bind(db.display_df_predictions, timestep=timestep_widget, model=model_widget)

button_get_predictions = pn.widgets.Button(name="Get predictions", button_type="primary")

configuration_box = pn.WidgetBox('## Select configuration', timestep_widget, model_widget, button_get_predictions)

def result(clicked):
    if clicked:
        return predictions()


df_predictions = pn.bind(result, button_get_predictions)


# Add filters to the df_predictions
filters = {
    'True Class': {'type': 'list', 'func': 'in', 'valuesLookup': True, 'sort': 'asc', 'multiselect': True},
    'Predicted Class': {'type': 'list', 'func': 'in', 'valuesLookup': True, 'sort': 'asc', 'multiselect': True},
}

filter_table = pn.widgets.Tabulator(
    df_predictions, pagination='local', layout='fit_columns', page_size=10, sizing_mode='stretch_width',
    header_filters=filters)

filename, button_download = filter_table.download_menu(
    text_kwargs={'name': 'Enter filename', 'value': 'predictions.csv'},
    button_kwargs={'name': 'Download table'})

download_file_box = pn.WidgetBox('## Download table', filename, button_download)


tab2 = pn.Column(
    pn.layout.Divider(),
    pn.Row(configuration_box), 
    filter_table,
    pn.Row(download_file_box),
    pn.layout.Divider(),
)


#**********************************
# Tab 3: getting numerical results
# *********************************
button_summary_results = pn.widgets.Button(name="Get Summary of Results", button_type="primary")
button_display_charts = pn.widgets.Button(name="Get plots", button_type="primary")

summary_results = pn.bind(db.summary_results)
pie_chart_true_labels = pn.bind(db.display_pie_charts, label="true")
pie_chart_predicted = pn.bind(db.display_pie_charts, label="predicted")


def result(clicked):
    if clicked:
        return summary_results()

def result2(clicked):
    if clicked:
        return pie_chart_true_labels()

def result3(clicked):
    if clicked:
        return pie_chart_predicted()

df_results = pn.bind(result, button_summary_results)
display_pie_chart_true = pn.Column(pn.bind(result2, button_display_charts), name="True Labels")
display_pie_chart_predicted = pn.Column(pn.bind(result3, button_display_charts), name="Predicted Labels")

# Create the Accordion with named sections
accordion_pie_charts = pn.Accordion(
    ("True Labels", display_pie_chart_true),
    ("Predicted Labels", display_pie_chart_predicted)
)

tab3 = pn.Column(
    pn.layout.Divider(),
    pn.Row(button_summary_results),
    df_results,
    pn.Row(button_display_charts),
    pn.Row(accordion_pie_charts),
    pn.layout.Divider(),
)


#****************************************************************************
# Tab 4: create visualizations of the original graph and the predicted graph
#****************************************************************************
button_create_original_graph = pn.widgets.Button(name="Display original graph", button_type="primary", sizing_mode='stretch_width')
button_create_predicted_graph = pn.widgets.Button(name="Display predicted graph", button_type="primary", sizing_mode='stretch_width')
button_graphs = pn.widgets.Button(name="Visualize graphs", button_type="primary", sizing_mode='stretch_width')

create_original_graph = pn.bind(db.display_graph, label="true")
create_predicted_graph = pn.bind(db.display_graph, label="predicted")

def result(clicked):
    if clicked:
        return create_original_graph()

def result2(clicked):
    if clicked:
        return create_predicted_graph()

original_graph = pn.Column(pn.bind(result, button_graphs), name="Original graph")
predicted_graph = pn.Column(pn.bind(result2, button_graphs), name="Predicted graph")

accordion_graphs = pn.Accordion(original_graph, predicted_graph)

tab4 = pn.Column(
    pn.layout.Divider(),
    pn.Row(button_graphs),
    pn.Row(accordion_graphs),
    pn.layout.Divider(),
)



# Unificate all the characteristics into a panel
panel_graph = pn.Column(pn.Row(pn.pane.Markdown('# ANOMALY DETECTION IN GRAPHS')),
                        pn.Tabs(('Predictions', tab2), ('Results', tab3),('Visualizations', tab4)))
panel_graph

NotificationArea(design=<class 'panel.theme.materi...)

Column(design=<class 'panel.theme.materi...)
    [0] Row(design=<class 'panel.theme.materi...)
        [0] Markdown(str, design=<class 'panel.theme.materi...)
    [1] Tabs(design=<class 'panel.theme.materi...)
        [0] Column(design=<class 'panel.theme.materi...)
            [0] Divider(design=<class 'panel.theme.materi...)
            [1] Row(design=<class 'panel.theme.materi...)
                [0] WidgetBox(design=<class 'panel.theme.materi...)
                    [0] Markdown(str, design=<class 'panel.theme.materi...)
                    [1] IntSlider(design=<class 'panel.theme.materi..., end=49, name='TimeStep', start=1, value=30)
                    [2] RadioButtonGroup(design=<class 'panel.theme.materi..., name='Models', options=['Random Forest', ...], value='Random Forest')
                    [3] Button(button_type='primary', design=<class 'panel.theme.materi..., name='Get predictions')
            [2] Tabulator(design=<class 'panel.theme.materi..., header_filters={'True Class': {'type': 'l...}, layout='fit_columns', page_size=10, pagination='local', sizing_mode='stretch_width')
            [3] Row(design=<class 'panel.theme.materi...)
                [0] WidgetBox(design=<class 'panel.theme.materi...)
                    [0] Markdown(str, design=<class 'panel.theme.materi...)
                    [1] TextInput(design=<class 'panel.theme.materi..., name='Enter filename', value='predictions.csv')
                    [2] Button(design=<class 'panel.theme.materi..., name='Download table')
            [4] Divider(design=<class 'panel.theme.materi...)
        [1] Column(design=<class 'panel.theme.materi...)
            [0] Divider(design=<class 'panel.theme.materi...)
            [1] Row(design=<class 'panel.theme.materi...)
                [0] Button(button_type='primary', design=<class 'panel.theme.materi..., name='Get Summary of Results')
            [2] ParamFunction(function, _pane=Str, defer_load=False, design=<class 'panel.theme.materi...)
            [3] Row(design=<class 'panel.theme.materi...)
                [0] Button(button_type='primary', design=<class 'panel.theme.materi..., name='Get plots')
            [4] Row(design=<class 'panel.theme.materi...)
                [0] Accordion(design=<class 'panel.theme.materi...)
                    [0] Column(design=<class 'panel.theme.materi..., name='True Labels')
                        [0] ParamFunction(function, _pane=Str, defer_load=False, design=<class 'panel.theme.materi...)
                    [1] Column(design=<class 'panel.theme.materi..., name='Predicted Labels')
                        [0] ParamFunction(function, _pane=Str, defer_load=False, design=<class 'panel.theme.materi...)
            [5] Divider(design=<class 'panel.theme.materi...)
        [2] Column(design=<class 'panel.theme.materi...)
            [0] Divider(design=<class 'panel.theme.materi...)
            [1] Row(design=<class 'panel.theme.materi...)
                [0] Button(button_type='primary', design=<class 'panel.theme.materi..., name='Visualize graphs', sizing_mode='stretch_width')
            [2] Row(design=<class 'panel.theme.materi...)
                [0] Accordion(design=<class 'panel.theme.materi...)
                    [0] Column(design=<class 'panel.theme.materi..., name='Original graph')
                        [0] ParamFunction(function, _pane=Str, defer_load=False, design=<class 'panel.theme.materi...)
                    [1] Column(design=<class 'panel.theme.materi..., name='Predicted graph')
                        [0] ParamFunction(function, _pane=Str, defer_load=False, design=<class 'panel.theme.materi...)
            [3] Divider(design=<class 'panel.theme.materi...)

In [334]:
def load_file(file):
    if file is not None:
        df = pd.read_csv(io.BytesIO(file))
        return df

file_input = pn.widgets.FileInput()
print(file_input.value)
button_load = pn.widgets.Button(name="Load file", button_type='primary')
# button_load.param.watch(interactions.load_file(), 'clicks')
file_loaded = pn.param.ParamFunction(pn.bind(load_file, file=file_input), loading_indicator=True)

upload_file_box = pn.WidgetBox('## Upload file', file_input, button_load)

pn.Column(
    pn.layout.Divider(),
    pn.Row(upload_file_box),
    file_loaded,
    pn.layout.Divider(),
)

None


Column(design=<class 'panel.theme.materi...)
    [0] Divider(design=<class 'panel.theme.materi...)
    [1] Row(design=<class 'panel.theme.materi...)
        [0] WidgetBox(design=<class 'panel.theme.materi...)
            [0] Markdown(str, design=<class 'panel.theme.materi...)
            [1] FileInput(design=<class 'panel.theme.materi...)
            [2] Button(button_type='primary', design=<class 'panel.theme.materi..., name='Load file')
    [2] ParamFunction(function, _pane=Str, defer_load=False, design=<class 'panel.theme.materi...)
    [3] Divider(design=<class 'panel.theme.materi...)

In [342]:
file_input = pn.widgets.FileInput()

file_input

FileInput(design=<class 'panel.theme.materi...)

In [350]:
if file_input.value is not None:
    print(file_input.filename)
    print(file_input.value)
    file = file_input.value
    preview_bytes = file[:100].decode('utf-8', errors='ignore')
    print("File preview (first 100 bytes):", preview_bytes)
    df = pd.read_csv(io.BytesIO(file))
    display(df)
else:
    print("failed")

train_set.csv
b''
File preview (first 100 bytes): 


EmptyDataError: No columns to parse from file

In [224]:
# Instantiate the template with widgets displayed in the sidebar
template = pn.template.MaterialTemplate(
    title='Anomaly Detection in Graphs',
    sidebar=["Upload file", "Predictions", "Results", "Visualizations"],
)
# Append a layout to the main area, to demonstrate the list-like API
template.main.append(
    panel_graph
)

template.show()

Launching server at http://localhost:56820


AssertionError: 

Coses que falten:
- Aplicar un tema diferent
- Ficar-li models GNN i AE
- Canviar el color dels botons

In [356]:
import panel as pn

pn.extension(notifications=True)

pn.state.notifications.error('This is an error notification.', duration=1000)
pn.state.notifications.info('This is a info notification.', duration=2000)

pn.state.notifications.warning('This is a warning notification.', duration=4000)

success = pn.state.notifications.success('This is a success notification.', duration=0)
pn.state.notifications.send('Fire!!!', background='red', icon='<i class="fas fa-burn"></i>');

NotificationArea(design=<class 'panel.theme.materi..., notifications=[Notification(_destroyed=F...])